In [1]:
import timeit
import math
import random
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def getCrossProduct(p0,p1,p2):
    # Returns which is more counterclockwise
    cross_product = (p1[0] - p0[0]) * (p2[1]-p0[1]) - (p2[0]-p0[0]) * (p1[1]-p0[1])

    if cross_product == 0:
        return 0 #p1
    elif cross_product > 0:
        return 1 #p2
    else:
        return -1 #p1

In [3]:
def findFurthestOld(origin, points):

    maxDist = 0
    maxPoint = None
    for point in points:
        dist = ((origin[0]-point[0])**2 + (origin[1]-point[1])**2)
        if dist > maxDist:
            maxPoint = point
            maxDist = dist
    return maxPoint


In [4]:
def findFurthest(origin, points):
    def distance(point):
        return math.sqrt((origin[0] - point[0]) ** 2 + (origin[1] - point[1]) ** 2)
    
    return max(points, key=distance, default=None)

In [5]:
def jarvismarch(inputSet):

        #ADD YOUR CODE HERE
        firstPoint = min(inputSet)
        curPoint = firstPoint
        outputSet = []

        nextPoint = None
        while nextPoint != firstPoint:
            nextPoint = firstPoint

            for cord in inputSet:
                if curPoint == nextPoint:
                    nextPoint = cord
                    continue

                if getCrossProduct(curPoint,nextPoint,cord) > 0:
                    nextPoint = cord
                            
            outputSet.append(nextPoint)
            curPoint = nextPoint

        return outputSet

In [6]:
def grahamscan(inputSet):
    outputSet = []
    pointAngles = {}
    firstPoint = min(inputSet, key=lambda p: (p[1], p[0])) # Get the lowest and leftmost point

    # Grouping the points by polar angle
    for point in inputSet:
        if point == firstPoint:
            continue
        
        angle_rad = math.atan2(point[1] - firstPoint[1], point[0] - firstPoint[0] )

        if angle_rad in pointAngles:
            pointAngles[angle_rad].append(point)
        else:
            pointAngles[angle_rad] = [point]

    tempSet = [subArr[1][0] if len(subArr) == 1 else findFurthest(firstPoint, subArr[1]) for subArr in sorted(list(pointAngles.items()))] + [firstPoint]
    
    for point in tempSet:
        while len(outputSet) >= 2 and getCrossProduct(outputSet[-2], outputSet[-1], point) < 0:
            outputSet.pop()  # Remove the last point in the outputSet because it makes a non-left turn
        outputSet.append(point)

    return outputSet
        


    

In [7]:
def chen(inputSet):
    outputSet = []
    groupSize = 4
    
    for i in range(1, len(inputSet)):
        groupSize = 2 ** 2 ** i
        subPoints = [inputSet[i:i + groupSize] for i in range(0, len(inputSet), groupSize)]
        subHulls = [grahamscan(points) for points in subPoints]
        allSubHullPoints = [point for subHull in subHulls for point in subHull]

        outputSet = jarvismarch(allSubHullPoints)
        if len(outputSet) <= groupSize:
            return outputSet

    return outputSet



In [8]:
class TestDataGenerator():
    def __init__(self):
        self.xPoints = []
        self.yPoints = []
        self.points = []
        self.jarvisBorder = []
        self.grahamBorder = []
        self.chenBorder = []
        self.dataSet =[]
        self.dataSetIndex = ["random","colinear","onHull"]
        # self.inititaliseBorders()
        # self.displayHull()

    def generatePoints(self, size):
        self.xPoints, self.yPoints, self.points = [],[],[]
        while len(self.xPoints) < size :
            xp = random.randint(0,32767)
            yp = random.randint(0,32767)
            if (xp,yp) not in self.points:
                self.xPoints.append(xp)
                self.yPoints.append(yp)
                self.points.append((xp,yp))
        self.dataSet.append(self.points)
        

    def generateColinear(self, size):
        self.xPoints, self.yPoints, self.points = [],[],[]
        for i in range(size):
            self.yPoints.append(0)
            self.xPoints.append(i)
            self.points.append((i,0))
        self.dataSet.append(self.points)

    def generatePointsOnConvexHull(self, num_points, center=(0, 0), radius=2000):
         """ Generate points evenly spaced along the circumference of a circle.
           Parameters: num_points (int): Number of points to generate.
           center (tuple): Coordinates of the center of the circle.
            Default is (0, 0). 
            radius (float): Radius of the circle. 
            Default is 100. 
            Returns: list: List of (x, y) coordinates representing the points. """ 
         self.xPoints, self.yPoints, self.points = [],[],[]
         for i in range(num_points):
             angle = 2 * math.pi * i / num_points 
             x = center[0] + radius * math.cos(angle) 
             y = center[1] + radius * math.sin(angle) 
             self.xPoints.append(x) 
             self.yPoints.append(y) 
             self.points.append((x, y))
         self.dataSet.append(self.points)

    def testCase4(self, size): # use this to create further test cases
        self.xPoints, self.yPoints, self.points = [],[],[]
        #insert code: should be varying h(points on hull) and n(total points)
        self.dataSet.append(self.points)
    
                
    def inititaliseBorders(self):
        self.initialiseJarvis()
        self.initialiseGraham()
        self.initialiseChen()

    def runJarvis(self, points):
        start = timeit.default_timer()
        self.jarvisBorder = jarvismarch(points)
        end = timeit.default_timer()
        return (end - start), len(self.jarvisBorder)
    
    def runGraham(self, points):
        start = timeit.default_timer()
        self.grahamBorder = grahamscan(points)
        end = timeit.default_timer()
        return (end - start), len(self.grahamBorder)
    
    def runChen(self, points):
        start = timeit.default_timer()
        self.chenBorder = chen(points)
        end = timeit.default_timer()
        return (end - start), len(self.chenBorder)


    def initialiseJarvis(self):
        self.jarvisBorder = jarvismarch(self.points)
        
        
    def initialiseGraham(self):
        self.grahamBorder = grahamscan(self.points)

    def initialiseChen(self):
        self.chenBorder = chen(self.points)

    def runTest(self, size):
        self.generatePoints(size)
        self.generatePointsOnConvexHull(size)
        self.generateColinear(size)
        self.inititaliseBorders()
        #elf.displayAllHull()

    def getDataSets(self):
        return self.dataSet, self.dataSetIndex

    def displayAllHull(self):
    
        # plt.figure(figsize=(3, 1))
        boundary = 4000

        # Create subplots with 1 row and 2 columns
        fig, axs = plt.subplots(1,3, figsize = (10,3))
        axs[0].scatter(self.xPoints, self.yPoints, color='red', s=5)
        axs[0].set_title("Jarvis")
        tempArr = self.jarvisBorder + [self.jarvisBorder[0]]
        x1, y1 = zip(*tempArr)
        axs[0].plot(x1, y1, '-', color = 'blue')
        axs[0].set_xlim(min(self.xPoints)-boundary, max(self.xPoints) + boundary)
        axs[0].set_ylim(min(self.yPoints)-boundary, max(self.yPoints) + boundary)

        axs[1].scatter(self.xPoints, self.yPoints, color='red', s=5)
        axs[1].set_title("Graham")
        tempArr = self.grahamBorder + [self.grahamBorder[0]]
        x2, y2 = zip(*tempArr)
        axs[1].plot(x2,y2, '-', color = 'blue')
        axs[1].set_xlim(min(self.xPoints)-boundary, max(self.xPoints) + boundary)
        axs[1].set_ylim(min(self.yPoints)-boundary, max(self.yPoints) + boundary)

        axs[2].scatter(self.xPoints, self.yPoints, color='red', s=5)
        axs[2].set_title("Chen")
        tempArr = self.chenBorder + [self.chenBorder[0]]
        x3, y3 = zip(*tempArr)
        axs[2].plot(x3,y3, '-', color = 'blue')
        axs[2].set_xlim(min(self.xPoints)-boundary, max(self.xPoints) + boundary)
        axs[2].set_ylim(min(self.yPoints)-boundary, max(self.yPoints) + boundary)

        plt.tight_layout()

        plt.show() 
    
    def displayHull(self, points):
        # plt.plot(xpoints, ypoints, 'ro--')

        # plt.plot(xpoints, ypoints, 'o', color='red')


        plt.scatter(self.xPoints, self.yPoints, color='red', s=5)

        # for (x, y) in plot.points:
        #     plt.annotate(f'({x}, {y})', (x, y), textcoords="offset points", xytext=(5,5), ha='center', fontsize=5)

        # plt.plot(xpoints, ypoints, '-', color = 'blue')
        tempArr = self.points + [self.points[0]]
        xline, yline = zip(*tempArr)
        plt.plot(xline, yline, '-', color = 'blue')

        # plt.xlim(-15000, 50000)
        # plt.ylim(-15000, 50000)
        plt.xlim(min(self.xPoints)-10000, max(self.xPoints) + 10000)
        plt.ylim(min(self.yPoints)-10000, max(self.yPoints) + 10000)
        plt.show()
        



In [9]:
import timeit
import matplotlib.pyplot as plt

class ExperimentalFramework():
  
  def __init__(self, dataSet, dataSetIndex):
    self.dataSet = dataSet
    self.dataSetIndex = dataSetIndex
    #contains data from all datasets make keys to identify

  def plotExecutionTimeAndHullPoints(self, testDataGenerator, maxPoints, step):
    # Initialize lists to store number of points, corresponding execution times, and hull points
    temp= []
    algo = ["(Jarvis)", "(Graham)", "(Chen)"]
    numPoints = [[] for _ in range(3)]
    executionTimes = [[] for _ in range(3)]
    hullPoints = [[] for _ in range(3)]
    
    # Generate test data and run Jarvis march for different numbers of points
    #0-jarvis 1-graham 2-chen
    for a in range(3):
      for size in range(2,maxPoints + 1, step):
        totalExecutionTime = 0
        totalHullSize = 0
        n = 10
        for i in range(n):  # Run n times for the same size
          testDataGenerator.generatePoints(size) # replace with other test case function
          if a == 0:
            executionTime, hullSize = testDataGenerator.runJarvis(testDataGenerator.points)
          elif a == 1:
            executionTime, hullSize = testDataGenerator.runGraham(testDataGenerator.points)
          else:
            executionTime, hullSize = testDataGenerator.runChen(testDataGenerator.points)
          totalExecutionTime += executionTime
          totalHullSize += hullSize
        averageExecutionTime = totalExecutionTime / n  # Calculate average execution time
        avgHullSize = totalHullSize / n

        numPoints[a].append(size)
        executionTimes[a].append(averageExecutionTime)
        hullPoints[a].append(avgHullSize)
    
    # Plot execution time against number of points
    
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    for a in range(3):
      plt.plot(numPoints[a], executionTimes[a], label = algo[a])
    plt.title('Execution Time against Total Number of Points')
    plt.xlabel('Number of Points')
    plt.ylabel('Execution Time (seconds)')
    plt.legend()
    plt.grid(True)

    # Plot number of points on hull against number of points
    plt.subplot(1, 2, 2)
    for a in range(3):
      plt.scatter(hullPoints[a], executionTimes[a], label=algo[a], marker='.')
    plt.title('Time against Points on Hull')
    plt.xlabel('Number of Points on Hull')
    plt.ylabel('Execution Time (seconds)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
      

In [10]:
test = TestDataGenerator()
 # Generate colinear points
size = 10000
for i in range(1):
    test.runTest(size)
    pass
  
experiment = ExperimentalFramework(test.dataSet, test.dataSetIndex)

experiment.plotExecutionTimeAndHullPoints(test, size, 1000)
